In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Диаграмма Вороного

## План конспекта
- [Определение](#definition)
- [Свойства](#properties)
- [Связь с триангуляцией Делоне](#delone)
- [Инкрементальный алгоритм построения диаграммы Вороного](#incremental_algorithm)
- [Диаграмма Вороного высшего порядка](#higher-order)
- [Алгоритм построения диаграммы Вороного высшего порядка](#higher-order_algorithm)
- [Полезные материалы](#usefull_links)

<a id='definition'></a>
## Определение диаграммы Вороного

### Описание
Диаграмма Вороного (англ. Voronoi diagram) конечного множества точек $P$ на плоскости представляет такое разбиение плоскости, при котором каждая область этого разбиения образует множество точек, более близких к одному из элементов множества $P$, чем к любому другому элементу множества.
Названа в честь российского учёного Георгия Феодосьевича Вороного. Также известна как: мозаика Вороного, разбиение Вороного, разбиение Дирихле.

### Формальное определение
Пусть $P = \{ p_1, p_2, ..., p_n\}$ — множество точек на плоскости.

**Определение**<br>
>Точка $p_i \in P$ называется ***сайтом (site)***.

**Определение**<br>
>***Ячейка Вороного (Voronoi cell, $\mathcal{V}(p_i)$)*** — множество точек плоскости $q$ таких, что для фиксированного сайта $p_i$ и любых других сайтов $p_j \in P, \ j \neq i$ верно неравенство $\rho(q, p_i) < \rho(q, p_j)$.

**Определение**<br>
>***Диаграмма Вороного (Voronoi diagram, $Vor(P)$)*** для сайтов $P = \{ p_1, p_2, ..., p_n\}$ на плоскости — это разбиение плоскости на ячейки Вороного для каждого сайта из $P$.

<img src="definition.png">

<a id='properties'></a>
## Свойства диаграммы Вороного

### Связь с пересечением полуплоскостей
Возьмём две точки плоскости: $p$ и $q$. Проведём серединный перпендикуляр к отрезку $pq$; полученную полуплоскость, которая содержит в себе $p$, обозначим $h(p, q)$, другую — $h(q, p)$. Заметим, что для точки $r$ выполняется $r \in h(p, q)$ тогда и только тогда, когда $\rho(r, p) < \rho(r, q)$. Отсюда вытекает следующее утверждение:<br>
**Утверждение**<br>
>$\mathcal{V}(p_i) = \bigcap\limits_{1 \leqslant j \leqslant n, j \neq i} h(p_i, p_j)$

Отсюда получаем, что ячейка Вороного — это пересечение $n - 1$ полуплоскостей, и поэтому представляет собой (возможно, неограниченную) открытую выпуклую область с не более чем $n - 1$ вершинами и $n - 1$ рёбрами.

### Топология диаграммы Вороного
**Теорема**<br>
>Пусть $P$ — множество из $n$ сайтов. Если они все лежат на одной прямой, то $Vor(P)$ представляет собой $n - 1$ параллельную прямую. Иначе $Vor(P)$ связная и все её рёбра — либо отрезки, либо лучи.

**Доказательство**<br>
>В случае, если все сайты лежат на одной прямой, каждая пара соседних сайтов порождает серединный перпендикуляр к отрезку, содержащему их, и, соответственно, к прямой, которая содержит все сайты. Так получаются $n - 1$ прямая, каждая из которых перпендикулярна прямой, содержащей сайты, а значит, эти прямые параллельны.

>Рассмотрим теперь случай, когда сайты не лежат на одной прямой. Покажем, что рёбра — это отрезки или лучи, от противного. Предположим, что есть ребро $e$, являющееся прямой. Пусть оно — граница ячеек $\mathcal{V}(p_i)$ и $\mathcal{V}(p_j)$. Пусть точка $p_k \in P$ не лежит на прямой $p_i p_j$ (по условию такая точка существует). Тогда серединный перпендикуляр к $p_j p_k$ не параллелен $e$, и, значит, он его пересекает. Но тогда та часть $e$, что лежит в $h(p_k, p_j)$, не может быть границей $\mathcal{V}(p_j)$, потому что она ближе к $p_k$, чем к $p_j$. Пришли к противоречию.

>Докажем теперь, что диаграмма связна. Предположим, что это не так. Тогда на её рёбрах найдутся две точки $s$ и $t$, между которыми нет пути по рёбрам диаграммы. Рассмотрим отрезок $st$. Он пересекает некоторое количество ячеек диаграммы. Пусть он пересекает какую-то ячейку в точках $a$ и $b$. От точки $a$ до точки $b$ можно добраться по рёбрам тогда и только тогда, когда ячейка связна. Раз пути из $s$ в $t$ нет, то какая-то из ячеек, пересекаемых отрезком $st$, несвязная. Это возможно, только если она представляет собой полосу, ограниченную двумя параллельными прямыми. Но в нашем случае в диаграмме не может быть прямых, пришли к противоречию.

### Размер структуры
**Теорема**<br>
>Для $n \geqslant 3$ сайтов диаграмма Вороного содержит не больше $2n - 5$ вершин и $ 3n - 6$ рёбер.

**Доказательство**<br>
>Для случая сайтов, лежащих на одной прямой, утверждение напрямую следует из вида диаграммы для этого случая, поэтому рассмотрим общий случай. По формуле Эйлера $v - e + f = 2$, где $v$ — число вершин, $e$ — число рёбер и $f$ — число граней связного планарного графа. Мы не можем сразу применить эту формулу к $Vor(P)$, потому что в этом графе есть полубесконечные рёбра. Поэтому добавим вершину $v_\infty$, и все полубесконечные рёбра мы превратим в рёбра, инцидентные ей. Таким образом мы увеличили число вершин на одну, а число рёбер не изменилось. Число граней равно $n$ по определению диаграммы Вороного. Тогда по формуле Эйлера получаем $(v + 1) - e + n = 2$.
Сумма степеней всех вершин полученного графа равна $2e$, так как у каждого ребра есть ровно два конца (нет петель). Также из каждой вершины исходят как минимум три ребра. Отсюда получаем $2e \geqslant 3 (v + 1)$.
Домножим равенство на два и вычтем из него полученную нижнюю границу для $2 \cdot e$, в результате получим $ v \leqslant 2n - 5$. Далее подставим этот результат в равенство и получим $e \leqslant 3n - 6$, что и требовалось доказать.

<a id='delone'></a>
## Связь с триангуляцией Делоне

**Определение**<br>
>***Наибольшая пустая окружность*** точки $q$ по отношению к $P$ (***largest empty circle*** of $q$ with respect to $P$, $C_P(q)$) — наибольшая окружность с центром в $q$ такая, что во внутренности соответствующего ей круга не лежит ни одного сайта из $P$.

**Лемма**<br>
>Точка $q$ — вершина диаграммы Вороного в том и только в том случае, когда $C_P(q)$ содержит три и более сайтов на своей границе.

**Доказательство**<br>
>Предположим, что $q$ существует, а $p_i, \ p_j, \ p_k$ — соответствующие точки. Так как внутри $C_P(q)$ нет других сайтов, а $q$ равноудалена от точек $p_i, \ p_j, \ p_k$, $q$ должна быть на границе $\mathcal{V}(p_i), \ \mathcal{V}(p_j), \ \mathcal{V}(p_k)$ одновременно, то есть вершиной диаграммы.
Докажем в другую сторону: каждая вершина $q$ диаграммы инцидентна минимум трём рёбрам, и, поэтому, как минимум трём ячейкам $\mathcal{V}(p_i), \ \mathcal{V}(p_j), \ \mathcal{V}(p_k)$. Тогда $q$ лежит на равном расстоянии от $p_i, \ p_j, \ p_k$ и не может быть другого сайта ближе к $q$, так как иначе $\mathcal{V}(p_i), \ \mathcal{V}(p_j), \ \mathcal{V}(p_k)$ не сойдутся в $q$. Поэтому можно построить окружность с центром в $q$ и $p_i, \ p_j, \ p_k$ на границе так, что внутри не будет других сайтов.

**Лемма**<br>
>Серединный перпендикуляр к отрезку $p_i p_j$ образует ребро диаграммы Вороного в том и только в том случае, если на нём есть точка $q$ такая, что $C_P(q)$ содержит на своей границе только сайты $p_i, \ p_j$.

**Доказательство**<br>
>Предположим, что $q$ существует. Тогда, так как $C_P(q)$ не содержит в себе сайтов и содержит $p_i, \ p_j$ на границе, $ \rho(q, p_i) = \rho(q, p_j) \leqslant \rho(q, p_k), \ 1 \leqslant k \leqslant n$. Отсюда выходит, что $q$ — вершина $Vor(P)$ или лежит на ребре диаграммы. Но по предыдущей лемме выходит, что $q$ не может быть вершиной диаграммы. Значит, она лежит на ребре, заданном серединным перпендикуляром к $p_i p_j$.

>Докажем в другую сторону: пусть серединный перпендикуляр к $p_i p_j$ задаёт ребро диаграммы. Наибольшая пустая окружность любой точки $q$ на этом ребре должна содержать на границе $p_i$ и $p_j$ (так как $q$ равноудалена от $p_i$ и $p_j$). Также эта окружность не должна содержать никаких других сайтов на границе, так как тогда она является вершиной.

**Теорема**<br>
>Если соединить все сайты, соответствующие смежным ячейкам диаграммы Вороного, получится триангуляция Делоне этого множества точек.

**Доказательство**<br>
>Если ячейки, соответствующие сайтам $p_i, \ p_j$, смежны, то серединный перпендикуляр к отрезку $p_i p_j$ образует ребро диаграммы Вороного, то есть к нему применима предыдущая лемма и можно построить окружность с $p_i$ и $p_j$ на границе, внутри которой не будет других сайтов. Триангуляции Делоне принадлежат [[Триангуляция Делоне#Критерий Делоне для рёбер|те и только те]] рёбра (с поправкой на точки, лежащие на одной окружности), на которых можно построить такую окружность, что внутри неё не будет лежать никаких точек. Тогда ребро $p_i p_j$ является ребром триангуляции Делоне. За счёт равносильности в обеих используемых леммах мы добавим все рёбра и не построим лишних.

<a id='incremental_algorithm'></a>
## Инкрементальный алгоритм построения диаграммы Вороного

Храним диаграмму в РСДС. Пусть у нас уже есть диаграмма для точек $p_1, p_2, ..., p_i$. Добавим новый сайт $p_{i+1}$. Сначала найдём сайт $p_j$, в ячейку которого попадает $p_{i+1}$, перебором. После этого строим новую ячейку: сначала проведём серединный перпендикуляр для $p_{i+1}p_j$, он пересечёт границу ячейки $\mathcal{V}(p_j)$ с ячейкой $\mathcal{V}(p_k)$; на следующем шаге будем строить серединный перпендикуляр для $p_{i+1} p_k$ и так далее.

В процессе построения перпендикуляров необходимо обновлять РСДС. Каждый раз, когда новое полуребро $e$, порождаемое $p_{i+1}$ и $p_j$, пересекает существовавшее ранее полуребро $e'$, создаётся новая вершина $v$ и начинается новое полуребро $e+1$.

Обновление РСДС происходит следующим образом:

* создаём вершину $v$ с полуребром $e$;
* для полуребра $e$ в РСДС второй конец в вершине $v$, следующее полуребро — $e'$, инцидентные грани — слева $\mathcal{V}(i+1)$, справа — $\mathcal{V}(j)$;
* добавляем в РСДС полуребро $e + 1$ с началом в $v$ и предыдущим полуребром $e$;
* удаляем все полурёбра, лежащие между вершиной начала $e$ и вершиной конца $e$, по часовой стрелке;
* обновляем полуребро, соответствующее грани для $\mathcal{V}(p_j)$ — им становится $e$.

Каждый шаг выполняется за $O(i)$, значит, суммарно диаграмма из $n$ сайтов с нуля создаётся за $O(n^2)$.

<img src="incremental.png">

<a id='higher-order'></a>
## Диаграмма Вороного высшего порядка

**Определение**<br>
>***Ячейка Вороного $k$-го порядка*** ($\mathcal{V}_k(p_1, p_2, ..., p_k)$) — множество точек, имеющих в качестве ближайших $k$ соседей множество сайтов $p_1, p_2, ..., p_k$.

<a id='higher-order_algorithm'></a>
## Алгоритм построения диаграммы Вороного высшего порядка

Чтобы построить диаграмму $k$-го порядка, возьмём диаграмму $k - 1$-го порядка. Каждая ячейка построена для некоторого набора $k-1$ сайтов. Обозначим множество этих сайтов за $S$. Пересечём каждую из этих ячеек с ячейками диаграммы первого порядка, построенной на множестве сайтов $P \setminus S$. Когда мы пересекаем ячейку $k-1$-го порядка для точек $S$ с ячейкой первого порядка для точки $p_i$, получаем ячейку для множества $S \cup \{p_i\}$. После пересечения ячеек необходимо объединить те, которые отвечают за одинаковый набор сайтов (это могут быть только соседние по ребру ячейки).

Итого совершаем $k$ шагов, на каждом строим $O(n)$ диграмм Вороного за время $O(n^3)$, пересекаем $O(n)$ ячеек с $O(n)$ ячейками за $O(n)$ времени, а потом объединяем ячейки за $O(n)$ (линейное количество соседних рёбер ячейки, а объединение происходит за $O(1)$ за счёт структуры РСДС). Итого $O(k \cdot n^3)$.

<img src="higher-order.png">

<a id='usefull_links'></a>
## Полезные материалы

- [Визуализатор диаграммы Вороного (вставка точки)](https://bl.ocks.org/mbostock/raw/4060366/) 
- [Ещё один визуализатор диаграммы Вороного](http://alexbeutel.com/webgl/voronoi.html)
- [Различные реализации построения диаграммы Вороного](https://rosettacode.org/wiki/Voronoi_diagram)
- [Подробная статья про диаграмму Вороного](http://www.pi6.fernuni-hagen.de/downloads/publ/tr198.pdf)